In [12]:
import init
import pandas as pd
import tensorflow

pd.set_option('display.max_columns', 500)

In [34]:
def score(p1, p2):
    d = {1:0, 2:0}
    for _ in range(100):
        winner = init.main(100, 0, [p1, p2])
        d[winner] += 1

    win_rate = d[1]/100
    return win_rate

In [29]:
print(score(1, 'Default'))

0.504


In [30]:
print(score(0.8, 'Default'))

0.623


In [35]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense

# Define the architecture of the neural network
model = Sequential()
model.add(Dense(10, input_dim=1, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam')

# Generate some training data
X_train = np.random.uniform(0, 1, size=(1000, 1))
y_train = np.array([score(x, 'Default') for x in X_train])

# Train the neural network
model.fit(X_train, y_train, epochs=100, batch_size=32)

# Use the neural network to predict the input that maximizes the output of score()
def maximize_score():
    x = np.random.uniform(0, 1, size=(1, 1))
    for i in range(100):
        grad = model.predict(x) * (1 - model.predict(x)) * 2 * (score(x, 'Default') - 0.5)
        x += grad
        x = np.clip(x, 0, 1)
    return x[0][0]

Epoch 1/100
32/32 [==============================] - 1s 1ms/step - loss: 0.6933
Epoch 2/100
32/32 [==============================] - 0s 1ms/step - loss: 0.6932
Epoch 3/100
32/32 [==============================] - 0s 1ms/step - loss: 0.6932
Epoch 4/100
32/32 [==============================] - 0s 2ms/step - loss: 0.6932
Epoch 5/100
32/32 [==============================] - 0s 2ms/step - loss: 0.6932
Epoch 6/100
32/32 [==============================] - 0s 2ms/step - loss: 0.6932
Epoch 7/100
32/32 [==============================] - 0s 3ms/step - loss: 0.6931
Epoch 8/100
32/32 [==============================] - 0s 2ms/step - loss: 0.6931
Epoch 9/100
32/32 [==============================] - 0s 2ms/step - loss: 0.6931
Epoch 10/100
32/32 [==============================] - 0s 1ms/step - loss: 0.6931
Epoch 11/100
32/32 [==============================] - 0s 2ms/step - loss: 0.6931
Epoch 12/100
32/32 [==============================] - 0s 2ms/step - loss: 0.6931
Epoch 13/100
32/32 [=================

In [57]:
print(score(.87, 'Default'))
print(score(.88, 'Default'))
print(score(.89, 'Default'))

0.62
0.62
0.6


In [46]:
res = [(i, score(i/100, 'Default')) for i in range(1, 100)]
res.sort(key = lambda x: -x[1])

print(res[0])


(88, 0.75)


In [27]:
print(maximize_score())

1/1 [==============================] - 0s 25ms/step
0.43276569528122966


In [45]:
import numpy as np

# Generate a range of inputs
X_test = np.linspace(0, 1, 100).reshape(-1, 1)

# Predict the output for each input
y_test = model.predict(X_test)

# Find the input that produces the highest output
x_best = X_test[np.argmax(y_test)]
y_best = np.max(y_test)

print("The input that produces the highest output is:", x_best[0])
print("The highest output is:", y_best)

4/4 [==============================] - 0s 2ms/step
The input that produces the highest output is: 0.3535353535353536
The highest output is: 0.50339764


In [63]:
def gradient_descent(p2, guess):
    def objective(x):
            return -model.predict([x])

    # Define the initial input, the learning rate, and the number of iterations
    x = guess
    learning_rate = 0.01
    num_iterations = 50

    # Define the beta parameters for the Adam optimizer
    beta1 = 0.9
    beta2 = 0.999

    # Define the initial values for the first and second moments
    m = 0
    v = 0

    iter_vals = []

    # Optimize the input using Adam optimization
    for i in range(num_iterations):
        if len(iter_vals) >= 2 and abs(iter_vals[-1] - iter_vals[-2]) < 0.0005:
             break

        # Compute the gradient of the objective function using numerical differentiation
        gradient = (objective(x + 0.001) - objective(x)) / 0.001
        # Change the sign of the gradient to maximize the objective function
        #gradient = -gradient
        # Update the first and second moments
        m = beta1 * m + (1 - beta1) * gradient
        v = beta2 * v + (1 - beta2) * gradient**2
        # Compute the bias-corrected first and second moments
        m_hat = m / (1 - beta1**(i+1))
        v_hat = v / (1 - beta2**(i+1))
        # Update the input using the Adam optimizer
        x = x - learning_rate * m_hat / (v_hat**(1/2) + 1e-8)

        iter_vals.append(x)

        print(x)

    return x, score(x, p2)

In [65]:
print(gradient_descent('Default', 0.8))

1/1 [==============================] - 0s 42ms/step
[[0.79]]
1/1 [==============================] - 0s 36ms/step
[[0.78000003]]
1/1 [==============================] - 0s 41ms/step
[[0.77000004]]
1/1 [==============================] - 0s 50ms/step
[[0.76000005]]
1/1 [==============================] - 0s 36ms/step
[[0.7500015]]
1/1 [==============================] - 0s 48ms/step
[[0.7400039]]
1/1 [==============================] - 0s 39ms/step
[[0.7300069]]
1/1 [==============================] - 0s 40ms/step
[[0.7200103]]
1/1 [==============================] - 0s 38ms/step
[[0.710014]]
1/1 [==============================] - 0s 42ms/step
[[0.7000144]]
1/1 [==============================] - 0s 46ms/step
[[0.6900123]]
1/1 [==============================] - 0s 18ms/step
[[0.6800118]]
1/1 [==============================] - 0s 43ms/step
[[0.6700164]]
1/1 [==============================] - 0s 38ms/step
[[0.6600175]]
1/1 [==============================] - 0s 53ms/step
[[0.65001565]]
1/1 [=======

In [104]:
def gradient_descent(p2, guess):
    def objective(x):
            return score(x, p2)

    # Define the initial input, the learning rate, and the number of iterations
    x = guess
    learning_rate = 0.01
    num_iterations = 50


    iter_vals = []

    # Optimize the input using Adam optimization
    for i in range(num_iterations):
        if len(iter_vals) >= 2 and abs(iter_vals[-1] - iter_vals[-2]) < 0.0005:
             break

        # Compute the gradient of the objective function using numerical differentiation
        gradient = (objective(x + 0.001) - objective(x)) / 0.001
       
        x += learning_rate * gradient

        iter_vals.append(x)

        print(x)

    return x, score(x, p2)

In [105]:
print(gradient_descent('Default', 0.8))

1.2800000000000005
1.0600000000000014
1.3200000000000005


KeyboardInterrupt: 